**Import Libraries**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow as tf

from keras.applications import VGG16

import cv2 as cv

import zipfile

from keras import models,layers
import os,shutil
from keras.preprocessing.image import ImageDataGenerator 
from keras import optimizers,metrics

from collections import Counter


from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.advanced_activations import LeakyReLU 
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [0]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
# #@title Default title text
# ! pip install pydrive
# # these classes allow you to request the Google drive API
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive 
# from google.colab import auth 
# from oauth2client.client import GoogleCredentials

# # 1. Authenticate and create the PyDrive client.
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

In [0]:
# id = '1QOYfgC1_jVV5ONh5Qqfzp1PNhhZDIcqV'
# downloaded = drive.CreateFile({'id': id})
# downloaded.GetContentFile('dataset52bd6ce.zip')

In [0]:
# listed = drive.ListFile({'q': "title contains '.zip' and 'root' in parents"}).GetList()
# for file in listed:
#   print('title {}, id {}'.format(file['title'], file['id']))

In [0]:
# import zipfile
# zip_ref = zipfile.ZipFile('dataset52bd6ce.zip', 'r')
# zip_ref.extractall('../')
# zip_ref.close()

In [0]:
# os.mkdir('../dataset/Validation')

In [0]:
# for i in range(15):
#   os.mkdir('../dataset/Validation/'+str(i)) 
  

In [0]:
# for i in os.listdir('../dataset/train/'):
#   all_files = [j for j in os.listdir('../dataset/train/'+i)]
#   print(i)
#   for j in range(int(len(all_files)*.15)):
#     src,dst = '../dataset/train/'+i+'/'+all_files[j],'../dataset/Validation/'+i+'/'+all_files[j]
#     shutil.move(src,dst)
  
    

In [0]:
for i in os.listdir('../dataset'):
  print(i)

Validation
train
sample_submissioin.csv
test


**Making Model**

In [0]:
input_shape = (100,100,1)
model = models.Sequential()


model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

# Fully connected layer
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.6))
model.add(Dense(15))

model.add(Activation('softmax'))



In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 98, 98, 32)        320       
_________________________________________________________________
batch_normalization_1 (Batch (None, 98, 98, 32)        128       
_________________________________________________________________
activation_1 (Activation)    (None, 98, 98, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 96, 96, 32)        9248      
_________________________________________________________________
batch_normalization_2 (Batch (None, 96, 96, 32)        128       
_________________________________________________________________
activation_2 (Activation)    (None, 96, 96, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 48, 32)        0         
__________

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(),
              metrics=['acc'])

**Train Data**

In [0]:
x_train, y_train = [], []
for i in os.listdir('../dataset/train'):
  for j in os.listdir('../dataset/train/'+i):
    x_train.append(cv.resize(cv.imread('../dataset/train/'+i+'/'+j,cv.IMREAD_GRAYSCALE),(100,100)))
    y_train.append(int(i))
    
x_train, y_train = np.array(x_train),np.array(y_train)
x_train.shape,y_train.shape

((52925, 100, 100), (52925,))

**Validation data**

In [0]:
x_test, y_test = [], []
for i in os.listdir('../dataset/Validation'):
  for j in os.listdir('../dataset/Validation/'+i):
    x_test.append(cv.resize(cv.imread('../dataset/Validation/'+i+'/'+j,cv.IMREAD_GRAYSCALE),(100,100)))
    y_test.append(int(i))
    
x_test, y_test = np.array(x_test),np.array(y_test)
x_test.shape,y_test.shape

((9333, 100, 100), (9333,))

**Test Data**

In [0]:
test = np.array([cv.resize(cv.imread('../dataset/test/'+i,cv.IMREAD_GRAYSCALE),(100,100)) for i in os.listdir('../dataset/test')])

test.shape

(21273, 100, 100)

In [0]:
gc.collect()

484

In [0]:
x_train = x_train.reshape(52925, 100, 100, 1)
x_test = x_test.reshape(9333, 100, 100, 1)
test = test.reshape(21273, 100, 100, 1)

In [0]:
from keras.utils import to_categorical,normalize

In [0]:
train_labels = to_categorical(y_train)
validation_labels = to_categorical(y_test)

In [0]:
gc.collect()

0

In [0]:
model.fit(x_train, train_labels, epochs=76, batch_size=512,validation_data=(x_test, validation_labels))

Train on 52925 samples, validate on 9333 samples
Epoch 1/76
52925/52925 [==============================] - 106s 2ms/step - loss: 2.4861 - acc: 0.2422 - val_loss: 2.2536 - val_acc: 0.2744
Epoch 2/76
52925/52925 [==============================] - 99s 2ms/step - loss: 2.0270 - acc: 0.3449 - val_loss: 2.6920 - val_acc: 0.2240
Epoch 3/76
52925/52925 [==============================] - 99s 2ms/step - loss: 1.6895 - acc: 0.4448 - val_loss: 2.2639 - val_acc: 0.3057
Epoch 4/76
 9728/52925 [====>.........................] - ETA: 1:14 - loss: 1.3051 - acc: 0.5774

52925/52925 [==============================] - 99s 2ms/step - loss: 1.3313 - acc: 0.5652 - val_loss: 2.3764 - val_acc: 0.3270
Epoch 5/76
45568/52925 [========================>.....] - ETA: 12s - loss: 0.9331 - acc: 0.7047

In [0]:
model.save('../dataset/CNN_VGG_1.h5')

In [0]:
del x_train,x_test,train_labels,validation_labels

NameError: ignored

In [0]:
gc.collect()

In [0]:
sub = pd.DataFrame()
sub['image_name'] = [i for i in os.listdir('..dataset/test')]

In [0]:
sub.head()
# ,category

In [0]:
sub['category'] = model.predict_classes(test)

In [0]:
sub.to_csv('../dataset/submission_vgg.csv',index=False)

In [0]:
from google.colab import files
files.download('../dataset/submission_vgg.csv')